In [5]:
import os
import sys
import json
from datetime import datetime
import subprocess
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..', '..'))
sys.path.append(root_dir)
import Birdeye.Basics.Master_Functions as Master_Functions

# Generating Data

In [4]:
# Specify the path to the Python file you want to run
python_file_path = 'Dataset_Generator_New_Tokens.py'

# Run the Python file
subprocess.run(['python', python_file_path])

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}

--------------------------------

Running on: solana

--------------------------------

NEW TOKEN FILTERS:

Min Liquidity: 10K
Max Liquidity: 100K
Min Market Cap: 100K
Max Market Cap: 300K
Filtering New Tokens for: 0 days, 1 hours, 5 minutes back
Token Launch Liquidity Filter: 10K

--------------------------------
 
 
--------------------------------

https://dexscreener.com/solana/FAVwsjCnEvSDTCJXmvyeSBX3RrYH8dTYSyFQH9SApump - Market Cap: 263K
https://dexscreener.com/solana/AFupKyMR75kZ2V6ooCwQHpBtLr6hAt866KWaMUrgpump - Market Cap: 219K
https://dexscreener.com/solana/2wAq5vooDHLD2ccrowpurKYhSQXAzQuct43m7rkpgSmz - Market Cap: 207K
https://dexscreener.com/solana/8ExunNtjVV8xWTkYRBXbYznC4UWqLR62BMEbf2EDpump - Market Cap: 188K
https://dexscreener.com/solana/9StrGbWhX8nuKSuTNjv5B3BgfLNrbk3c3Hh9YB85pump - Market Cap: 100K

Token summary data saved to: Data/New_Token_Data/2024_10_

CompletedProcess(args=['python', 'Dataset_Generator_New_Tokens.py'], returncode=0)

# Import Data

In [27]:
# Get the most recent folder
base_path = 'Data/New_Token_Data'
current_date = datetime.now().strftime('%Y_%m_%d')
date_folder = os.path.join(base_path, current_date)
ohlcv_folder = os.path.join(date_folder, 'OHLCV_Data')
ohlcv_datetime_folder = Master_Functions.get_most_recent_folder(ohlcv_folder)

# Print the name of the folder being used for import
print(f"Importing OHLCV data from folder: {ohlcv_datetime_folder}")

# Import the OHLCV data
imported_ohlcv_data = Master_Functions.import_ohlcv_data(ohlcv_datetime_folder)

# Print the structure of the imported OHLCV data dictionary
# print("\nStructure of imported OHLCV data:")
# ohlcv_structure = {token: {timeframe: df.shape for timeframe, df in timeframes.items()} for token, timeframes in imported_ohlcv_data.items()}
# print(json.dumps(ohlcv_structure, indent=2))

# Example of how to access the data
print("\nExample of accessing the data:")
for token_address, timeframes in imported_ohlcv_data.items():
    print(f"Token: {token_address}")
    for timeframe, df in timeframes.items():
        print(f"  Timeframe: {timeframe}")
        print(f"  Data shape: {df.shape}")
        #print(f"  First few rows:")
        #print(df.head())
        print("\n")
    break  # Only print the first token as an example

# Example of calculating simple metrics
def calculate_metrics(df):
    if df.empty:
        return {
            'last_price': None,
            'daily_return': None,
            'volatility': None
        }
    return {
        'last_price': df['close'].iloc[-1],
        'daily_return': (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100 if len(df) > 1 else 0,
        'volatility': df['close'].pct_change().std() * 100 if len(df) > 1 else 0
    }

# Calculate metrics for each token and timeframe
metrics = {}
for token_address, timeframes in imported_ohlcv_data.items():
    metrics[token_address] = {}
    for timeframe, df in timeframes.items():
        metrics[token_address][timeframe] = calculate_metrics(df)

# Display metrics
print("\nCalculated metrics:")
for token_address, timeframes in metrics.items():
    print(f"Token: {token_address}")
    for timeframe, metric in timeframes.items():
        print(f"  Timeframe: {timeframe}")
        if metric['last_price'] is not None:
            print(f"    Last Price: {metric['last_price']:.4f}")
            print(f"    Daily Return: {metric['daily_return']:.2f}%")
            print(f"    Volatility: {metric['volatility']:.2f}%")
        else:
            print("    No data available")
    print("\n")

Importing OHLCV data from folder: Data/New_Token_Data/2024_10_22/OHLCV_Data/2024_Oct_22_0624AM

Example of accessing the data:
Token: GF6Zzthi8sAmg5gysQFB7LaT62sWh69kuDArB3MQMdoS
  Timeframe: 1m
  Data shape: (40, 7)


  Timeframe: 5m
  Data shape: (9, 7)


  Timeframe: 3m
  Data shape: (14, 7)


  Timeframe: 4H
  Data shape: (1, 7)


  Timeframe: 1H
  Data shape: (2, 7)


  Timeframe: 15m
  Data shape: (3, 7)


  Timeframe: 1D
  Data shape: (1, 7)


  Timeframe: 30m
  Data shape: (2, 7)



Calculated metrics:
Token: GF6Zzthi8sAmg5gysQFB7LaT62sWh69kuDArB3MQMdoS
  Timeframe: 1m
    Last Price: 0.0003
    Daily Return: 254.24%
    Volatility: 26.43%
  Timeframe: 5m
    Last Price: 0.0003
    Daily Return: 174.84%
    Volatility: 51.02%
  Timeframe: 3m
    Last Price: 0.0003
    Daily Return: 205.20%
    Volatility: 53.68%
  Timeframe: 4H
    Last Price: 0.0003
    Daily Return: 0.00%
    Volatility: 0.00%
  Timeframe: 1H
    Last Price: 0.0003
    Daily Return: -1.86%
    Volatility: nan

Fibonacci Strategy 

In [30]:
def fib_strategy(df_high_tf, df_low_tf, fib_levels, fib_ranges, take_profit):
    """
    Implement a Fibonacci retracement trading strategy with dynamic stop loss.
    
    :param df_high_tf: DataFrame containing high timeframe data
    :param df_low_tf: DataFrame containing low timeframe data
    :param fib_levels: List of Fibonacci retracement levels
    :param fib_ranges: List of price ranges for each Fibonacci level
    :param take_profit: Take profit percentage
    :return: List of executed trades
    """
    trades = []
    # Calculate the cumulative maximum (All-Time High) for the closing price
    ath = df_high_tf['close'].cummax()
    
    # Iterate through each row in the high timeframe DataFrame
    for index, row in df_high_tf.iterrows():
        current_ath = ath.loc[index]
        
        # Check each Fibonacci level
        for level, range_width in zip(fib_levels, fib_ranges):
            # Calculate the Fibonacci price and its range
            fib_price = current_ath * (1 - level)
            fib_price_range = (fib_price * (1 - range_width), fib_price * (1 + range_width))
            
            # Check if the current close price is at or below the Fibonacci price
            if row['close'] <= fib_price:
                # Filter low timeframe data starting from the current high timeframe datetime
                df_low_tf_sub = df_low_tf[df_low_tf['datetime'] >= row['datetime']]
                
                # Set the Entry Condition (EC) candle as the first candle in the filtered low timeframe data
                ec_candle = df_low_tf_sub.iloc[0]
                ec_range = (fib_price_range[0], fib_price_range[1])
                
                # Iterate through the filtered low timeframe data
                for idx, candle in df_low_tf_sub.iterrows():
                    # Check if the current candle's close price is within the Fibonacci price range
                    if ec_range[0] <= candle['close'] <= ec_range[1]:
                        # Calculate the price move for the EC candle and the current candle
                        ec_move = ec_candle['high'] - ec_candle['open']
                        candle_move = candle['high'] - candle['open']
                        
                        # Check for a bullish move (current candle move > 1.03 * EC candle move)
                        if candle_move > 1.03 * ec_move:
                            # Place a Buy Order
                            buy_price = 1.05 * ec_move  # 2% Slippage
                            
                            # Set Stop Loss to the lower bound of the Fibonacci price range
                            sl_price = fib_price_range[0]
                            
                            # Calculate Take Profit price
                            tp_price = buy_price * (1 + take_profit)
                            
                            # Initialize the trade dictionary
                            trade = {
                                'entry_time': candle['datetime'],
                                'entry_price': buy_price,
                                'stop_loss': sl_price,
                                'take_profit': tp_price,
                                'exit_time': None,
                                'exit_price': None,
                                'outcome': None,
                                'fib_level': level,
                                'fib_range': range_width
                            }
                            
                            # Simulate the trade using the remaining low timeframe data
                            trade = simulate_trade(df_low_tf_sub[idx:], trade)
                            
                            # Add the completed trade to the trades list
                            trades.append(trade)
                            
                            # Move to the next signal (break the inner loop)
                            break
    
    # Return the list of all executed trades
    return trades

Simulate Trade

In [31]:
def simulate_trade(df, trade):
    for idx, candle in df.iterrows():
        # Check Stop Loss
        if candle['low'] <= trade['stop_loss']:
            trade['exit_time'] = candle['datetime']
            trade['exit_price'] = trade['stop_loss']
            trade['outcome'] = 'loss'
            break
        # Check Take Profit
        elif candle['high'] >= trade['take_profit']:
            trade['exit_time'] = candle['datetime']
            trade['exit_price'] = trade['take_profit']
            trade['outcome'] = 'win'
            break
    else:
        # If neither SL nor TP is hit, exit at last price
        trade['exit_time'] = df.iloc[-1]['datetime']
        trade['exit_price'] = df.iloc[-1]['close']
        trade['outcome'] = 'hold'
    return trade

Running the Strategy Across Multiple Timeframes and Tokens

In [ ]:
# Import timeframes from Master_Functions
timeframes_imported = Master_Functions.timeframes
print(f"Available timeframes: {timeframes_imported}")

# Define the hyperparameter grid
take_profit_levels = [0.02, 0.04, 0.06]  # 2%, 4%, 6%
fib_levels = [0.382, 0.5, 0.618, 0.786]
fib_ranges = [0.05, 0.10, 0.15, 0.20]

# Define the range of timeframes to use as high timeframes
timeframe_range = ['5m', '15m', '30m', '1H', '4H', '1D']

# Results storage
all_trades = []

# Function to get the next lowest timeframe
def get_next_lowest_timeframe(current_tf, available_tfs):
    current_index = available_tfs.index(current_tf)
    if current_index > 0:
        return available_tfs[current_index - 1]
    return None

# Loop over tokens
for token_address, timeframes in imported_ohlcv_data.items():
    # Get available timeframes for this token
    available_tfs = sorted([tf for tf in timeframes.keys() if tf in timeframes_imported])
    
    # Loop over timeframes
    for high_tf in timeframe_range:
        if high_tf not in available_tfs:
            continue
        
        df_high_tf = timeframes[high_tf]
        
        # Get the next lowest timeframe
        low_tf = get_next_lowest_timeframe(high_tf, available_tfs)
        
        if low_tf is None:
            print(f"No lower timeframe available for {high_tf}. Skipping.")
            continue
        
        df_low_tf = timeframes[low_tf]
        
        print(f"Processing {token_address}: High TF = {high_tf}, Low TF = {low_tf}")
        
        # Loop over hyperparameters
        for tp in take_profit_levels:
            trades = fib_strategy(df_high_tf, df_low_tf, fib_levels, fib_ranges, tp)
            
            # Add token and parameters info to trades
            for trade in trades:
                trade['token'] = token_address
                trade['high_tf'] = high_tf
                trade['low_tf'] = low_tf
                trade['take_profit'] = tp
            all_trades.extend(trades)

print(f"Total trades generated: {len(all_trades)}")